In [1]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('DF').getOrCreate()
from pyspark.sql.functions import *
from pyspark.sql.types import *


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=809425f51db062a823039bafd15d187f82a2ab46f8e0b14a11a97d3bf6acedbe
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
df =spark.read.csv('/content/students.csv',header=True,inferSchema=True)

In [3]:
df.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

In [4]:
male_count = df.filter(col("gender") == "male").count()
female_count = df.filter(col("gender") == "female").count()
male_count

482

In [5]:
female_count

518

In [6]:
ethnicity_list = df.select("race/ethnicity").distinct().rdd.flatMap(lambda x: x).collect()
ethnicity_list

['group B', 'group C', 'group D', 'group A', 'group E']

In [7]:
education_levels = df.select("parental level of education").distinct().rdd.flatMap(lambda x: x).collect()
education_levels

['some high school',
 "associate's degree",
 'high school',
 "bachelor's degree",
 "master's degree",
 'some college']

In [8]:
female_highschool_above_79_math = df.filter((col("gender") == "female") & (col("parental level of education") == "high school") & (col("math score") > 79)).count()
female_highschool_above_79_math

5

In [9]:
average_math_male = df.filter(col("gender") == "male").agg(avg("math score").alias("avg_math_male")).collect()[0]["avg_math_male"]
average_math_female = df.filter(col("gender") == "female").agg(avg("math score").alias("avg_math_female")).collect()[0]["avg_math_female"]

if average_math_male > average_math_female:
  print("male avg is high :",average_math_male)

else:
  print("female avg is high :",average_math_female)

male avg is high : 68.72821576763485


In [10]:
average_reading_male = df.filter(col("gender") == "male").agg(avg("reading score").alias("avg_reading_male")).collect()[0]["avg_reading_male"]
average_reading_female = df.filter(col("gender") == "female").agg(avg("reading score").alias("avg_reading_female")).collect()[0]["avg_reading_female"]
average_reading_male

65.47302904564316

In [11]:
average_reading_female

72.60810810810811

In [12]:
education_score_dependency = df.groupBy("parental level of education").agg(avg("math score").alias("avg_math"), avg("reading score").alias("avg_reading"), avg("writing score").alias("avg_writing"))
education_score_dependency.show()

+---------------------------+------------------+-----------------+-----------------+
|parental level of education|          avg_math|      avg_reading|      avg_writing|
+---------------------------+------------------+-----------------+-----------------+
|           some high school|63.497206703910614|66.93854748603351|64.88826815642459|
|         associate's degree| 67.88288288288288|70.92792792792793| 69.8963963963964|
|                high school| 62.13775510204081|64.70408163265306|62.44897959183673|
|          bachelor's degree| 69.38983050847457|             73.0|73.38135593220339|
|            master's degree|  69.7457627118644|75.37288135593221|75.67796610169492|
|               some college|  67.1283185840708|69.46017699115045|68.84070796460178|
+---------------------------+------------------+-----------------+-----------------+



In [13]:
above_70_math_no_prep = df.filter((col("test preparation course") == "none") & (col("math score") > 70))
above_70_math_no_prep.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group C|               some college|    standard|                   none|        76|           78|           75|
|female|       group B|         associate's degree|    standard|                   none|        71|           83|           78|
|  male|       group C|                high school|    standard|                   none|        88|     